In [2]:
import json
from unskript import nbparams
from unskript.fwk.workflow import Task, Workflow
from unskript.secrets import ENV_MODE, ENV_MODE_LOCAL

env = {"ENV_MODE": "ENV_MODE_LOCAL"}
secret_store_cfg = {"SECRET_STORE_TYPE": "SECRET_STORE_TYPE_LOCAL"}

paramDict = {"cluster_name": "kjsbfkjwe", "namespace": "dg-ops", "region": "us-west-2"}
unSkriptOutputParamDict = {}
paramDict.update(env)
paramDict.update(secret_store_cfg)
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)
cluster_name = nbParamsObj.get('cluster_name')
namespace = nbParamsObj.get('namespace')
region = nbParamsObj.get('region')
w = Workflow(env, secret_store_cfg, None, global_vars=globals())

<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong>Rebuild a HA Cloud backend using an existing 'p' directory</strong></div>
</center>
<p>&nbsp;</p>
<p>&nbsp;</p>
<h2>Runbook Scenarios</h2>
<blockquote>
<p>These are the scenarios where this Runbook could be useful.</p>
</blockquote>
<ol>
<li>When more than 2 alpha nodes are down due to any error condition - <span style="color: rgb(224, 62, 45);">OOM, Log Compact</span> or any other <span style="color: rgb(224, 62, 45);">error condition</span></li>
<li>Data corruption on 2 alpha nodes</li>
<li>Manually performing a copy/clone operation between clusters</li>
<li>Data corruption on a non-HA cluster, where a 'p' directory is newer or better than any available backup</li>
<li>Copying data to new cluster for testing or root cause analysis</li>
</ol>

<h2 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h2>
<ol>
<li><a href="#Step-1" target="_self" rel="noopener">Set EKS Context</a></li>
<li><a href="#Step-2" target="_self" rel="noopener">Get Pods in the Given Namespace</a></li>
<li><a href="#Step-3" target="_self" rel="noopener">Get the max values from Zero Node</a></li>
<li><a href="#Step-4" target="_self" rel="noopener">Setting up alpha nodes with the right data in 'p' directories</a></li>
<li><a href="#Step-5" target="_self" rel="noopener">Cleanup all Zero pods on Current Cluster</a></li>
<li><a href="#Step-6" target="_self" rel="noopener">Scale down Alpha group to 0 replicas on active cluster</a></li>
<li><a href="#Step-7">Delete PVCs for non-leader alphas</a></li>
<li><a href="#Step-8">Scale down Zero group to 0 replicas on active cluster</a></li>
<li><a href="#Step-9">Scale up Zero group to 3 replicas on target namespace</a></li>
<li><a href="#Step-10">Assign UID, Transaction TS and Namespace IDs to Target Zero leader</a></li>
<li><a href="#Step-11">Scale the Alpha group up to just 1 replica. Alpha-0 should start up </a></li>
<li><a href="#Step-11" target="_self" rel="noopener">Wait for schema to lazy-load on Alpha-0 </a></li>
<li><a href="#Step-12" target="_self" rel="noopener">Wait for Alpha-0 to create a snapshot</a></li>
<li><a href="#Step-13" target="_self" rel="noopener">Scale up Alpha group to 2 replicas for Alpha-1 to start-up</a></li>
<li><a href="#Step-14" target="_self" rel="noopener">Verify that Alpha-1 is healthy </a></li>
<li><a href="#Step-15" target="_self" rel="noopener">Finally, Scale the Alpha group to 3 replicas </a></li>
<li><a href="#Step-16" target="_self" rel="noopener">Replicate Step-14 to confirm Snapshot stream to Alpha-2 was successful </a></li>
<li><a href="#Step-17" target="_self" rel="noopener">Post Recovery Verification Steps</a></li>
</ol>
<p>&nbsp;</p>
<p>You can use these hyper-links to nagivate to the section of the runbook.&nbsp;</p>

<h3><a id="Step-1" target="_self" rel="nofollow"></a>Set kubernetes context to the EKS Cluster which has the faulty backend</h3>
<div class="page" title="Page 1">
<div class="section">
<div class="layoutArea">
<div class="column">&nbsp;</div>
</div>
</div>
</div>
<blockquote>
<div class="page" title="Page 1">
<div class="section">
<div class="layoutArea">
<div class="column">
<p>&nbsp;</p>
</div>
</div>
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>aws  eks update-kubeconfig --name &lt;EKS-cluster-name&gt; --region &lt;Region-
Name&gt;
</pre>
</div>
</div>
</div>
<div class="section">
<div class="layoutArea">
<div class="column">
<pre><strong><span style="background-color: rgb(241, 196, 15); color: rgb(0, 0, 0);">Example for us-east-1:</span></strong><br><br>aws  eks update-kubeconfig --name us-east-1-prod-gaas-regional-eks --
region us-east-1
</pre>
</div>
</div>
</div>
</div>
</div>
</blockquote>

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_execute_cli_command_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_execute_cli_command(handle, aws_command: str) -> str:

    result = handle.aws_cli_command(aws_command)
    if result is None or result.returncode != 0:
        print(
            f"Error while executing command ({aws_command}): {result}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "awscreds",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "ee29f14b-31a6-44ed-b304-22f820cd1265"
}''')
task.configure(inputParamsJson='''{
    "aws_command": "f\\"aws eks update-kubeconfig --name {cluster_name} --region {region}\\""
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_execute_cli_command, lego_printer=aws_execute_cli_command_printer, hdl=hdl, args=args)

<h2 id="Get-All-Pods-belonging-to-the-Namespace"><a id="Step-2" target="_self" rel="nofollow"></a>Get All Pods belonging to the Namespace</h2>
<p>&nbsp;</p>
<blockquote>
<p>kubectl get pods -n &lt;namespace&gt;</p>
</blockquote>
<p>&nbsp;</p>
<p>Next, Find the leader Node from the Pods&nbsp;</p>
<p>&nbsp;</p>
<blockquote>
<div class="page" title="Page 1">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl exec -it zerostatefulset-0 -n &lt;namespace&gt; -- /bin/sh
curl -s localhost:6080/state | jq '{alphas: .groups."1".members[]|select
(.leader==true), zeros: .zeros[]|select(.leader==true)}'
</pre>
</div>
</div>
</div>
</div>
</blockquote>

In [109]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()


    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace} |  cut -d' ' -f 1\\""
    }''')
task.configure(outputName="all_pods")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

NAME
bulk01-dgraph-alpha-0
bulk01-dgraph-alpha-1
bulk01-dgraph-alpha-2
bulk01-dgraph-zero-0
bulk01-dgraph-zero-1
bulk01-dgraph-zero-2



In [110]:
zero_pod_name = '' # noqa
for pod in all_pods.split('\n'):
    if pod.find("zero") != -1:
        # Search the first Zero'th Pod and store it for further use
        zero_pod_name = pod
        break

# zero_pod_name = [x for x in pods if "zero" in x]
print(zero_pod_name)


jq_leader_cmd = ''' 
/bin/bash -c "curl -s localhost:6080/state | /usr/bin/jq '{alphas: .groups.\\"1\\".members[]|select(.leader==true), zeros: .zeros[]|select(.leader==true)}'"
'''  # noqa
jq_max_cmd = '''
/bin/bash -c "curl -s localhost:6080/state | /usr/bin/jq | grep '\\"max'"
'''  # noqa


bulk01-dgraph-zero-0


In [113]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl exec {zero_pod_name} -n {namespace} -- {jq_leader_cmd.strip()}\\""
    }''')
task.configure(outputName="json_leader_output")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

{
  "alphas": {
    "id": "1",
    "groupId": 1,
    "addr": "bulk01-dgraph-alpha-0.bulk01-dgraph-alpha-headless.dg-ops.svc.cluster.local:7080",
    "leader": true,
    "amDead": false,
    "lastUpdate": "1693566231",
    "learner": false,
    "clusterInfoOnly": false,
    "forceGroupId": false
  },
  "zeros": {
    "id": "1",
    "groupId": 0,
    "addr": "bulk01-dgraph-zero-0.bulk01-dgraph-zero-headless.dg-ops.svc.cluster.local:5080",
    "leader": true,
    "amDead": false,
    "lastUpdate": "0",
    "learner": false,
    "clusterInfoOnly": false,
    "forceGroupId": false
  }
}



In [114]:
import json
try:
    alpha_leader_node = json.loads(json_leader_output).get('alphas').get('addr').split('.')[0]
    zero_leader_node = json.loads(json_leader_output).get('zeros').get('addr').split('.')[0]
except Exception as e:
    print("ERROR: Extracting Zero and Alpha leader node Addr", e)

non_alpha_leader_nodes = []
non_zero_leader_nodes = []
all_alpha_nodes = []
all_zero_nodes = []

for pod in all_pods.split('\n'):
    if pod.find('alpha') != -1:
        if pod.strip() != alpha_leader_node.strip():
            non_alpha_leader_nodes.append(pod.strip())
    elif pod.find('zero') != -1:
        if pod.strip() != zero_leader_node.strip():
            non_zero_leader_nodes.append(pod.strip())

print("Alpha Leader Node: ", alpha_leader_node)
print("    Non Leader Alpha Nodes: ", non_alpha_leader_nodes)
print("Zero Leader Node:", zero_leader_node)
print("    Non Leader Zero Nodes: ", non_zero_leader_nodes)

# This would be needed for Step-4
all_alpha_nodes.append(alpha_leader_node)
for n in non_alpha_leader_nodes:
    all_alpha_nodes.append(n)
all_zero_nodes.append(zero_leader_node)
for n in non_zero_leader_nodes:
    all_zero_nodes.append(n)

Alpha Leader Node:  bulk01-dgraph-alpha-0
    Non Leader Alpha Nodes:  ['bulk01-dgraph-alpha-1', 'bulk01-dgraph-alpha-2']
Zero Leader Node: bulk01-dgraph-zero-0
    Non Leader Zero Nodes:  ['bulk01-dgraph-zero-1', 'bulk01-dgraph-zero-2']


<h2 id="Get-the-max-values-from-the-Zero-node---(maxUID,-maxTxnTs-and-maxNsID)-and-save-these-to-be-used-later"><a id="Step-3" target="_self" rel="nofollow"></a>Get the max values from the Zero node - (maxUID, maxTxnTs and maxNsID) and save these to be used later</h2>
<div class="page" title="Page 1">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre style="padding-left: 40px;">kubectl exec -it zerostatefulset-0 -n &lt;namespace&gt; -- /bin/sh curl -s localhost:6080/state | jq | grep '"max'
</pre>
<pre style="padding-left: 40px;">##Should return max values like the example below
"maxUID": "669546334271",
"maxTxnTs": "41060000",
"maxNsID": "41061000"
</pre>
</div>
</div>
</div>
</div>

In [115]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl exec {zero_leader_node} -n {namespace} -- {jq_max_cmd.strip()}\\""
    }''')
task.configure(outputName="jq_max_output")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

  "maxUID": "9800000",
  "maxTxnTs": "390000",
  "maxNsID": "0",
  "maxRaftId": "3",
    "maxNodes": "18446744073709551615",



In [116]:
import json

jq_max_result = {}
for line in jq_max_output.split('\n'):
    line = line.strip()
    k = line.split(':')[0]
    v = line.split(':')[-1]
    k = str(k).strip()
    v = str(v).strip('"')
    jq_max_result[k.strip('"')] = str(v.strip('"').strip(',')).strip(' ').strip('"')

try:
    max_uid=jq_max_result.get('maxUID')
    max_ts=jq_max_result.get('maxTxnTs')
    max_nsid=jq_max_result.get('maxNsID')
except Exception as e:
    print("ERROR: Cannot parse MaxUID / MaxTS/ MaxNSID", e)
    
print("MaxUID:", max_uid)
print("MaxTxnTs:", max_ts)
print("MaxNsID:", max_nsid)

MaxUID: 9800000
MaxTxnTs: 390000
MaxNsID: 0


<h2 id="Setting-up-alpha-nodes-with-the-right-data-in-'p'-directories"><a id="Step-4" target="_self" rel="nofollow"></a>Setting up alpha nodes with the right data in 'p' directories</h2>
<blockquote>
<div class="page" title="Page 1">
<div class="section">
<div class="layoutArea">
<div class="column">
<p>- If Alpha-0 and Zero-0 are the leaders (from Step 2)</p>
<ul>
<li>Rename the &lsquo;w&rsquo; and &lsquo;t&rsquo; directories on all Alphas and rename the &lsquo;p&rsquo; directories on just Alpha-1 and Alpha-2</li>
<li>Retain the &lsquo;p&rsquo; directory on Alpha-0 - This data is retained to rebuild the cluster</li>
</ul>
<p>- If Alpha-0 is not a leader</p>
<ul>
<li>Copy the &lsquo;p&rsquo; directory from good alpha to alpha-0.</li>
</ul>
</div>
</div>
<div class="layoutArea">
<div class="column">
<ul>
<li style="list-style-type: none;">
<ul>
<li>This can be done using kubectl copy</li>
</ul>
</li>
</ul>
</div>
</div>
</div>
</div>
<div class="page" title="Page 2">
<div class="layoutArea">
<div class="column">
<ul>
<li style="list-style-type: none;">
<ul>
<li>The good p dir has to be copied first to a local folder and then copied over to the alpha-0. NOTE: Bring up an EC2 instance in the&nbsp;same region as the cluster. The time and data transfer cost is very high if its done across regions.</li>
</ul>
</li>
<li>If its not possible to exec into the alpha-0 , then it has to be copied over to the pvc by doing node-shell into the node</li>
</ul>
<p>- In some situation, you might copy over a different p directory if we are replacing with an older or different data elsewhere)</p>
<p>&nbsp;</p>
<p>- Steps to get inside a alpha and renaming the directories:</p>
<div class="page" title="Page 2">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre style="padding-left: 40px;">kubectl exec -it alphastatefulset-0 -n &lt;namespace&gt; -- /bin/sh mv t t_&lt;date&gt;; mv w w_&lt;date&gt;
##Example-  mv t t_11272022 ; mv w w_11272022 )
mv p p_&lt;date&gt;; ### Do it only for the non-leader alpha
</pre>
</div>
</div>
</div>
</div>
</div>
</div>
</div>
</blockquote>

In [117]:
from datetime import datetime

current_time = datetime.now()
dir_name = current_time.strftime('%m%d%Y')
print("Generated Backup Directory Name: ", dir_name)
print("Constructed commands to delete existing pwt directories on respective pods")

# Helper function to form kubectl commands
def generate_cmd(pod, original_dir, new_dir):
    return f"kubectl exec -n {namespace} {pod} -- /bin/sh -c \"mkdir -p {new_dir} && mv {original_dir}/* {new_dir}/ && rmdir {original_dir}\""

# Backup W & T Directories on all Alpha Nodes
w_dir_mv_cmd = [generate_cmd(pod, "/dgraph/w", f"/dgraph/w_{dir_name}") for pod in all_alpha_nodes]
t_dir_mv_cmd = [generate_cmd(pod, "/dgraph/t", f"/dgraph/t_{dir_name}") for pod in all_alpha_nodes]

# Backup P Directory on Non Leader alpha Nodes
p_dir_mv_cmd = [generate_cmd(pod, "/dgraph/p", f"/dgraph/p_{dir_name}") for pod in non_alpha_leader_nodes]

Generated Backup Directory Name:  09012023


In [118]:
# Here we assume that Log Compact message if not seen on the Alpha node, then we 
# Declare that Alpha as a good alpha node
alpha_good_cmd = []
for pod in all_alpha_nodes: 
    cmd = f"kubectl logs {pod} -n {namespace} --tail=10000 | grep -i \"Log Compact\" | wc -l"
    print(cmd)
    alpha_good_cmd.append(cmd)


kubectl logs bulk01-dgraph-alpha-0 -n dg-ops --tail=10000 | grep -i "Log Compact" | wc -l
kubectl logs bulk01-dgraph-alpha-1 -n dg-ops --tail=10000 | grep -i "Log Compact" | wc -l
kubectl logs bulk01-dgraph-alpha-2 -n dg-ops --tail=10000 | grep -i "Log Compact" | wc -l


In [119]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "alpha_good_cmd",
    "iter_parameter": "kubectl_command"
    }''')
task.configure(outputName="good_alpha_node_log")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

Execution Summary: 
   Number of Execution(s): 3
   Iterator values:
        kubectl logs bulk01-dgraph-alpha-0 -n dg-ops --tail=10000 | grep -i "Log Compact" | wc -l		: ✓
        kubectl logs bulk01-dgraph-alpha-1 -n dg-ops --tail=10000 | grep -i "Log Compact" | wc -l		: ✓
        kubectl logs bulk01-dgraph-alpha-2 -n dg-ops --tail=10000 | grep -i "Log Compact" | wc -l		: ✓


In [120]:
index = 0
for item in good_alpha_node_log.items():
    if int(item[-1].strip()) == 0:
        good_alpha = all_alpha_nodes[index]
        break
    index += 1

print(f"Good Alpha : {good_alpha}")

Good Alpha : bulk01-dgraph-alpha-0


In [121]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "w_dir_mv_cmd",
    "iter_parameter": "kubectl_command"
    }''')

task.configure(conditionsJson='''{
    "condition_enabled": false,
    "condition_cfg": "alpha_leader_node[len(alpha_leader_node)-2:] == '-0'",
    "condition_result": true
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

Execution Summary: 
   Number of Execution(s): 3
   Iterator values:
        kubectl exec -n dg-ops bulk01-dgraph-alpha-0 -- /bin/sh -c "mkdir -p /dgraph/w_09012023 && mv /dgraph/w/* /dgraph/w_09012023/ && rmdir /dgraph/w"		: ✓
        kubectl exec -n dg-ops bulk01-dgraph-alpha-1 -- /bin/sh -c "mkdir -p /dgraph/w_09012023 && mv /dgraph/w/* /dgraph/w_09012023/ && rmdir /dgraph/w"		: ✓
        kubectl exec -n dg-ops bulk01-dgraph-alpha-2 -- /bin/sh -c "mkdir -p /dgraph/w_09012023 && mv /dgraph/w/* /dgraph/w_09012023/ && rmdir /dgraph/w"		: ✓


In [122]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "t_dir_mv_cmd",
    "iter_parameter": "kubectl_command"
    }''')

task.configure(conditionsJson='''{
    "condition_enabled": false,
    "condition_cfg": "alpha_leader_node[len(alpha_leader_node)-2:] == '-0'",
    "condition_result": true
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

Execution Summary: 
   Number of Execution(s): 3
   Iterator values:
        kubectl exec -n dg-ops bulk01-dgraph-alpha-0 -- /bin/sh -c "mkdir -p /dgraph/t_09012023 && mv /dgraph/t/* /dgraph/t_09012023/ && rmdir /dgraph/t"		: ✓
        kubectl exec -n dg-ops bulk01-dgraph-alpha-1 -- /bin/sh -c "mkdir -p /dgraph/t_09012023 && mv /dgraph/t/* /dgraph/t_09012023/ && rmdir /dgraph/t"		: ✓
        kubectl exec -n dg-ops bulk01-dgraph-alpha-2 -- /bin/sh -c "mkdir -p /dgraph/t_09012023 && mv /dgraph/t/* /dgraph/t_09012023/ && rmdir /dgraph/t"		: ✓


In [123]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "p_dir_mv_cmd",
    "iter_parameter": "kubectl_command"
    }''')

task.configure(conditionsJson='''{
    "condition_enabled": false,
    "condition_cfg": "alpha_leader_node[len(alpha_leader_node)-2:] == '-0'",
    "condition_result": true
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

Execution Summary: 
   Number of Execution(s): 2
   Iterator values:
        kubectl exec -n dg-ops bulk01-dgraph-alpha-1 -- /bin/sh -c "mkdir -p /dgraph/p_09012023 && mv /dgraph/p/* /dgraph/p_09012023/ && rmdir /dgraph/p"		: ✓
        kubectl exec -n dg-ops bulk01-dgraph-alpha-2 -- /bin/sh -c "mkdir -p /dgraph/p_09012023 && mv /dgraph/p/* /dgraph/p_09012023/ && rmdir /dgraph/p"		: ✓


<h2 id="Cleanup-all-Zero-Pods-on-Current-Active-Namespace"><a id="Step-5" target="_self" rel="nofollow"></a>Cleanup all Zero Pods on Current Source Namespace</h2>
<blockquote>
<div class="page" title="Page 2">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl exec -it zerostatefulset-0 -n &lt;namespace&gt; -- mv /dgraph/zw
/dgraph/zw_&lt;date&gt;
kubectl exec -it zerostatefulset-1 -n &lt;namespace&gt; -- mv /dgraph/zw
/dgraph/zw_&lt;date&gt;
</pre>
<pre>kubectl exec -it zerostatefulset-2 -n &lt;namespace&gt; -- mv /dgraph/zw
/dgraph/zw_&lt;date&gt;
</pre>
</div>
</div>
</div>
</div>
</blockquote>

In [125]:
# Helper function to form kubectl commands for moving directories
def generate_move_cmd(pod, original_dir, new_dir):
    return f"kubectl exec -n {namespace} {pod} -- /bin/sh -c \"mkdir -p {new_dir} && mv {original_dir}/* {new_dir}/ && rmdir {original_dir}\""

# Backup zw Directory on all Zero Nodes
zero_pods_cleanup_cmd = [generate_move_cmd(pod, "/dgraph/zw", f"/dgraph/zw_{dir_name}") for pod in all_zero_nodes]

print("Constructed zero pods cleaup command")


Constructed zero pods cleaup command


In [126]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "zero_pods_cleanup_cmd",
    "iter_parameter": "kubectl_command"
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

Execution Summary: 
   Number of Execution(s): 3
   Iterator values:
        kubectl exec -n dg-ops bulk01-dgraph-zero-0 -- /bin/sh -c "mkdir -p /dgraph/zw_09012023 && mv /dgraph/zw/* /dgraph/zw_09012023/ && rmdir /dgraph/zw"		: ✓
        kubectl exec -n dg-ops bulk01-dgraph-zero-1 -- /bin/sh -c "mkdir -p /dgraph/zw_09012023 && mv /dgraph/zw/* /dgraph/zw_09012023/ && rmdir /dgraph/zw"		: ✓
        kubectl exec -n dg-ops bulk01-dgraph-zero-2 -- /bin/sh -c "mkdir -p /dgraph/zw_09012023 && mv /dgraph/zw/* /dgraph/zw_09012023/ && rmdir /dgraph/zw"		: ✓


<h2 id="Scale-down-Alpha-nodes-on-Active-Cluster"><a id="Step-6" target="_self" rel="nofollow"></a>Scale down Alpha nodes on Source Cluster</h2>
<blockquote>
<div class="page" title="Page 2">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl scale --replicas=0 sts alphastatefulset -n &lt;namespace&gt;
</pre>
<pre># Check for pods to be gone with the following command
kubectl get pods -n &lt;namespace&gt; | grep alphastatefulset  ## Should
return no pods
</pre>
</div>
</div>
</div>
</div>
</blockquote>

In [127]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())

task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get sts -n {namespace} -o name | cut -d'/' -f 2\\""
    }''')
task.configure(outputName="source_sts_names")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

bulk01-dgraph-alpha
bulk01-dgraph-zero



In [128]:
source_alpha_sts = [x for x in source_sts_names.split('\n') if x.find('alpha') != -1]
source_zero_sts = [x for x in source_sts_names.split('\n') if x.find('zero') != -1]
# target_alpha_sts = [x for x in target_sts_names.split('\n') if x.find('alpha') != -1]
# target_zero_sts = [x for x in target_sts_names.split('\n') if x.find('zero') != -1]

print("Source Alpha STS :", source_alpha_sts)
print("Source Zero STS : ", source_zero_sts)
# print("Target Alpha STS: ", target_alpha_sts)
# print("Target Zero sts :", target_zero_sts)

Source Alpha STS : ['bulk01-dgraph-alpha']
Source Zero STS :  ['bulk01-dgraph-zero']


In [129]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl scale --replicas=0 sts {source_alpha_sts[0]} -n {namespace}\\""
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

statefulset.apps/bulk01-dgraph-alpha scaled



<h2 id="Scale-Down-Zero-Nodes-on-source-cluster">Delete PVCs for non-leader alphas</h2>
<p>This step makes sure that the actual data and cache for the non-leader alphas is deleted&nbsp;</p>

In [184]:
non_leader_alpha_pvc_names = [f"datadir-{node}" for node in non_alpha_leader_nodes]

kubectl_delete_pvc_command = f"kubectl delete pvc -n {namespace} {' '.join(non_leader_alpha_pvc_names)} --force"

print("Commands created to delete non-leader PVCs")


Commands created to delete non-leader PVCs


In [131]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from kubernetes.client.rest import ApiException


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String
        in case of Error.
    """
    if handle.client_side_validation is not True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)

    if result is None:
        print(
            f"Error while executing command ({kubectl_command}) (empty response)")
        return ""

    if result.stderr:
        raise ApiException(f"Error occurred while executing command {kubectl_command} {result.stderr}")

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "kubectl_delete_pvc_command"
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

Error while executing: (Error occurred while executing command kubectl delete pvc -n dg-ops datadir-bulk01-dgraph-alpha-1 datadir-bulk01-dgraph-alpha-2 --force warning: Immediate deletion does not wait for confirmation that the running resource has been terminated. The resource may continue to run on the cluster indefinitely.
)
Reason: None

Execution Failed: Error: (Error occurred while executing command kubectl delete pvc -n dg-ops datadir-bulk01-dgraph-alpha-1 datadir-bulk01-dgraph-alpha-2 --force warning: Immediate deletion does not wait for confirmation that the running resource has been terminated. The resource may continue to run on the cluster indefinitely.
)
Reason: None

  Task Parameters: {'kubectl_command': 'kubectl delete pvc -n dg-ops datadir-bulk01-dgraph-alpha-1 datadir-bulk01-dgraph-alpha-2 --force'}


ApiException: (Error occurred while executing command kubectl delete pvc -n dg-ops datadir-bulk01-dgraph-alpha-1 datadir-bulk01-dgraph-alpha-2 --force warning: Immediate deletion does not wait for confirmation that the running resource has been terminated. The resource may continue to run on the cluster indefinitely.
)
Reason: None


<h2 id="Scale-Down-Zero-Nodes-on-source-cluster"><a id="Step-7" target="_self" rel="nofollow"></a>Scale Down Zero Nodes on source cluster</h2>
<p>&nbsp;</p>
<blockquote>
<div class="page" title="Page 2">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl scale --replicas=0 sts zerostatefulset -n &lt;namespace&gt;
</pre>
<pre># Check for pods to be gone with the following command
kubectl get pods -n &lt;namespace&gt; | grep zerostatefulset &lt;-- Should
return no pods
</pre>
</div>
</div>
</div>
</div>
</blockquote>

In [132]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl scale --replicas=0 sts {source_zero_sts[0]} -n {namespace}\\""
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

statefulset.apps/bulk01-dgraph-zero scaled



In [135]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace} -o name | wc -l | xargs echo -n\\""
    }''')
task.configure(pollJson='''{
    "poll_check_output_type": "VALUE_TYPE_STR",
    "poll_check_output_value": "str(0)",
    "poll_step_interval": 5,
    "poll_timeout": 20,
    "poll_enabled": false
    }''')
task.configure(outputName="pods_after_scale_down")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

poll_enabled=False poll_check_output_type=<TaskPollValueTypeEnum.string: 'VALUE_TYPE_STR'> poll_check_output_value='str(0)' poll_step_interval=5 poll_timeout=20
0


In [185]:
if int(pods_after_scale_down) == 0:
    print("All Pods have been deleted")
else:
    print("Pods still scaling down, please wait...")
    print("To check the status, run the previous action again Verify All Pods are deleted")


All Pods have been deleted


<h2 id="Scale-up-Zero-group-to-3-replicas-on-target-namespace"><a id="Step-8" target="_self" rel="nofollow"></a>Scale up Zero group to 3 replicas.</h2>
<blockquote>
<div class="page" title="Page 3">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl scale --replicas=3 sts zerostatefulset -n &lt;namespace&gt;
</pre>
<pre># Check for all 3 pods to be up and ready  with the following command
kubectl get pods -n &lt;namespace&gt; | grep zerostatefulset
### Wait  for all  3 pods to be up and READY state 1/1
</pre>
</div>
</div>
</div>
</div>
</blockquote>

In [138]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl scale --replicas=3 sts {source_zero_sts[0]} -n {namespace}\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "int(pods_after_scale_down) == 0",
    "condition_result": true
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

statefulset.apps/bulk01-dgraph-zero scaled



In [145]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace} | grep 1/1 | grep -i zero | wc -l | xargs echo -n\\""
    }''')
task.configure(pollJson='''{
    "poll_check_output_type": "VALUE_TYPE_STR",
    "poll_check_output_value": "str(3)",
    "poll_enabled": false,
    "poll_step_interval": 5,
    "poll_timeout": 60
    }''')
task.configure(outputName="after_scale_up_zero_pods")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

poll_enabled=False poll_check_output_type=<TaskPollValueTypeEnum.string: 'VALUE_TYPE_STR'> poll_check_output_value='str(3)' poll_step_interval=5 poll_timeout=60
3


In [146]:
if int(after_scale_up_zero_pods) == 3:
    print("All zero pods successfully scaled up!")
else:
    print("Zero pods still scaling up, please wait...")
    print("To check the status, run the previous action again -Verify Zero group has been scaled to 3 replicas")

All zero pods successfully scaled up!


<h2 id="Assign-the-UIDs,-transaction-timestamps-and-namespace-Ids-to-the-Zero-endpoint-from-Step-3"><a id="Step-9" target="_self" rel="nofollow"></a>Assign the UIDs, transaction timestamps and namespace Ids to the Zero endpoint from Step-3</h2>
<ul>
<li>Find the leader Zero nodes in the cluster by following Step-2</li>
<li>Execute the assign endpoints on the leader zero with the values from Step-3</li>
</ul>
<blockquote>
<div class="page" title="Page 3">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl exec -it zerostatefulset-0 -n &lt;namespace&gt; &ndash;- /bin/sh
(above command is for zerostatefulset-0 is leader) -- Use the pod name
of the leader zero
</pre>
<pre>#Run the following Assign calls with values captured at step 3
curl "localhost:6080/assign?what=uids&amp;num=xxxx"
curl "localhost:6080/assign?what=timestamps&amp;num=xxxx"
curl "localhost:6080/assign?what=nsids&amp;num=xxxx"
</pre>
</div>
</div>
</div>
</div>
</blockquote>

In [147]:
# maxid_cmd = f'curl "localhost:6080/assign?what=uids&num={max_uid}"'
# maxts_cmd = f'curl "localhost:6080/assign?what=timestamps&num={max_ts}"'
# maxnsid_cmd = f'curl "localhost:6080/assign?what=nsids&num={max_nsid}"'

curl_cmds = [
    f'curl "localhost:6080/assign?what=uids&num={max_uid}"',
    f'curl "localhost:6080/assign?what=timestamps&num={max_ts}"',
    f'curl "localhost:6080/assign?what=nsids&num={max_nsid}"'
]

curl_command = f"kubectl exec {zero_leader_node} -n {namespace} -- sh -c '{{' && '.join(curl_cmds)}}'"

print(curl_command)


kubectl exec bulk01-dgraph-zero-0 -n dg-ops -- sh -c '{' && '.join(curl_cmds)}'


In [148]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "curl_command"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "int(after_scale_up_zero_pods) == 3",
    "condition_result": true
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h2 id="Scale-the-Alpha-Group-up-to-just-1-replica.-Alpha-0-Should-startup-on-Target-namespace"><a id="Step-10" target="_self" rel="nofollow"></a>Scale the Alpha Group up to just 1 replica. (Alpha-0 Should startup)&nbsp;</h2>
<blockquote>
<div class="page" title="Page 3">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl scale --replicas=1 sts alphastatefulset -n &lt;namespace&gt;
</pre>
</div>
</div>
</div>
</div>
</blockquote>

In [149]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl scale --replicas=1 sts {source_alpha_sts[0]} -n {namespace}\\""
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

statefulset.apps/bulk01-dgraph-alpha scaled



In [152]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace} | grep Running | wc -l | xargs echo -n\\""
    }''')
task.configure(pollJson='''{
    "poll_check_output_type": "VALUE_TYPE_STR",
    "poll_check_output_value": "str(4)",
    "poll_enabled": false,
    "poll_step_interval": 5,
    "poll_timeout": 60
    }''')
task.configure(outputName="scale_first_pod")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

poll_enabled=False poll_check_output_type=<TaskPollValueTypeEnum.string: 'VALUE_TYPE_STR'> poll_check_output_value='str(4)' poll_step_interval=5 poll_timeout=60
4


In [153]:
if int(scale_first_pod) == 4:
    print("First Alpha pod is scaled up!")
else:
    print("Alpha pod is still scaling up, please wait... ")
    print("Check the status again by running the previous action- Get Number of Pods that exist")

First Alpha pod is scaled up!


<h2 id="Wait-for-the-Schema-to-lazy-load-on-Alpha-0"><a id="Step-11" target="_self" rel="nofollow"></a>Wait for the Schema to lazy-load on Alpha-0</h2>
<ul>
<li>Check Grafana-Loki for progress</li>
<li>Look at the Logs as the schema lazy loads</li>
</ul>
<div class="page" title="Page 3">
<div class="section">
<div class="layoutArea">
<div class="column">
<blockquote>
<p>kubectl logs alphastatefulset-0 -n &lt;namespace&gt; -f<br><br></p>
</blockquote>
<ul>
<li>Wait until ready state becomes 1/1 for alphastatefulset-0 pod</li>
</ul>
<blockquote>
<div class="page" title="Page 3">
<div class="layoutArea">
<div class="column">
<p>IMPORTANT: This process could sometime take 30-45mins mins to complete if there are lot of predicates (like our shared clusters).</p>
</div>
</div>
</div>
</blockquote>
<div class="page" title="Page 3">
<div class="layoutArea">
<div class="column">
<p>&nbsp;</p>
<blockquote>
<div class="page" title="Page 3">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl get pods -n &lt;namespace&gt;
NAME                     READY   STATUS    RESTARTS
alphastatefulset-0       1/1     Running   0
</pre>
</div>
</div>
</div>
</div>
</blockquote>
</div>
</div>
</div>
</div>
</div>
</div>
</div>

<p class="p1">After Step-12 ,&nbsp; we need to run these commands to create a snapshot :</p>
<ol class="ol1">
<li class="li1">kubectl exec -ti pod/bulk01-dgraph-alpha-0 -n dgraph-test<span class="Apple-converted-space">&nbsp; </span>-- bash</li>
<li class="li1">dgraph increment --num=10000</li>
</ol>
<p class="p1">&nbsp;</p>
<p class="p1">&nbsp;</p>

<h2 id="Wait-for-Alpha-0-is-created-a-snapshot.-Check-logs-for-&#96;CreateSnapshot&#96;"><a id="Step-12" target="_self" rel="nofollow"></a>Wait for Alpha-0 is created a snapshot. Check logs for `CreateSnapshot`</h2>
<blockquote>
<div class="page" title="Page 4">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl logs alphastatefulset-0 -n &lt;namespace&gt; -f | grep
"CreateSnapshot"
</pre>
</div>
</div>
</div>
</div>
</blockquote>

In [155]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl logs {alpha_leader_node} -n {namespace} --tail=10000| grep 'Creating snapshot'\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "int(scale_first_pod) == 4",
    "condition_result": true
    }''')
task.configure(outputName="snapshot_log_message")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

I0901 17:47:02.376566      19 draft.go:627] Creating snapshot at Index: 5630, ReadTs: 5593



In [156]:
assert snapshot_log_message.find('Creating snapshot') != -1

# If not asserted, snapshot was created successfuly 
print("Alpha-0 successfully created snapshot")

Alpha-0 successfully created snapshot


<h2 id="Scale-up-Alpha-group-to-2-replicas"><a id="Step-13" target="_self" rel="nofollow"></a>Scale up Alpha group to 2 replicas</h2>
<blockquote>
<div class="page" title="Page 4">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl scale &ndash;-replicas=2 sts alphastatefulset -n &lt;namespace&gt;
</pre>
</div>
</div>
</div>
</div>
</blockquote>
<p>Check Alpha-0 logs for a "Snapshot Streaming" to ensure snapshot transfer happened</p>
<blockquote>
<div class="page" title="Page 4">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl logs alphastatefulset-0  -n &lt;namespace&gt; -f | grep -i "Snapshot
Streaming"
</pre>
</div>
</div>
</div>
</div>
</blockquote>

In [157]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl scale --replicas=2 sts {source_alpha_sts[0]} -n {namespace}\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "int(scale_first_pod) == 4",
    "condition_result": true
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

statefulset.apps/bulk01-dgraph-alpha scaled



In [161]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl logs -n {namespace} {alpha_leader_node} --tail=10000 | grep -i 'Stream snapshot: OK'\\""
    }''')
task.configure(outputName="log_streaming_output")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

I0901 17:49:58.878541      19 snapshot.go:300] Stream snapshot: OK



In [162]:
assert log_streaming_output.find("Stream snapshot: OK") != -1

print("Snapshot streaming successful!")

Snapshot streaming successful!


<h2 id="Verify-that-Alpha-1-is-Healthy"><a id="Step-14" target="_self" rel="nofollow"></a>Verify that two alphas are healthy<a class="jp-InternalAnchorLink" href="#Step-15">&para;</a></h2>
<p>&nbsp;</p>
<blockquote>
<div class="page" title="Page 4">
<div class="layoutArea">
<div class="column">
<p>After a snapshot stream is successful, confirm that the Alpha-1 is &lsquo;READY&rsquo; and &lsquo;Running&rsquo;. The schema load should be much faster on Alpha-1 on account of the snapshot stream. At this point, the cluster would start processing mutations successfully since quorum is reached.</p>
</div>
</div>
</div>
</blockquote>

In [167]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace} | grep 1/1 | grep alpha | wc -l | xargs echo -n\\""
    }''')
task.configure(pollJson='''{
    "poll_check_output_type": "VALUE_TYPE_STR",
    "poll_check_output_value": "str(2)",
    "poll_enabled": false,
    "poll_step_interval": 5,
    "poll_timeout": 30
    }''')
task.configure(outputName="second_alpha_pod")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

poll_enabled=False poll_check_output_type=<TaskPollValueTypeEnum.string: 'VALUE_TYPE_STR'> poll_check_output_value='str(2)' poll_step_interval=5 poll_timeout=30
2


In [174]:
if second_alpha_pod == '2':
    print("Successfully scaled up 2 alpha pods")
else:
    print("Please wait for the second alpha to scale up...\n")
    print("Run Step-15 to verify again")

Successfully scaled up 2 alpha pods


<h2 id="Finally,-scale-the-Alpha-group-to-3-replicas"><a id="Step-15" target="_self" rel="nofollow"></a>Finally, scale the Alpha group to 3 replicas</h2>
<p>&nbsp;</p>
<blockquote>
<div class="page" title="Page 4">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl scale &ndash;-replicas=3 sts alphastatefulset -n &lt;namespace&gt;
</pre>
</div>
</div>
</div>
</div>
</blockquote>

In [175]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl scale --replicas=3 sts {source_alpha_sts[0]} -n {namespace}\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "int(second_alpha_pod) == 2",
    "condition_result": true
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

statefulset.apps/bulk01-dgraph-alpha scaled



<h2 id="Repeat-Step-14"><a id="Step-16" target="_self" rel="nofollow"></a>Repeat Step 14</h2>
<p>&nbsp;</p>
<div class="page" title="Page 4">
<div class="layoutArea">
<div class="column">
<p>Verify that all 3 alpha and zero pods are running and are in ready state is 1/1</p>
<blockquote>
<div class="page" title="Page 4">
<div class="section">
<div class="layoutArea">
<div class="column">
<pre>kubectl get pods -n &lt;namespace&gt; | grep statefulset
</pre>
<pre># Results Should look like below
alphastatefulset-0    1/1     Running
alphastatefulset-1    1/1     Running
alphastatefulset-2    1/1     Running
zerostatefulset-0     1/1     Running
zerostatefulset-1     1/1     Running
zerostatefulset-2     1/1     Running
</pre>
</div>
</div>
</div>
</div>
</blockquote>
</div>
</div>
</div>

In [181]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "k8sss",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "16972f66-a07a-4730-93fb-3d35250c4fc5"
}''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace} | awk '/1\\\\/1/ {{ count++ }} END {{ print count }}'\\""
    }''')
task.configure(pollJson='''{
    "poll_check_output_type": "VALUE_TYPE_NUMBER",
    "poll_check_output_value": "6",
    "poll_enabled": false,
    "poll_step_interval": 5,
    "poll_timeout": 60
    }''')
task.configure(outputName="alpha_and_zero_pod_count")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

poll_enabled=False poll_check_output_type=<TaskPollValueTypeEnum.number: 'VALUE_TYPE_NUMBER'> poll_check_output_value=6 poll_step_interval=5 poll_timeout=60
6



In [182]:
if int(alpha_and_zero_pod_count) == 6:
    print("Alpha and Zero Pods are all up!")
else:
    print("Please wait for the all pods to start...\n")
    print("Run Step-16 to verify again")


Alpha and Zero Pods are all up!


<h2 id="Post-Recovery-Verification"><a id="Step-17" target="_self" rel="nofollow"></a>Post Recovery Verification</h2>
<ul>
<li>Check Grafana to ensure that <span style="color: rgb(22, 145, 121);"><strong>Max Transaction Ts </strong></span>and <span style="color: rgb(22, 145, 121);"><strong>RAFT Applied Index</strong></span> start rising</li>
<li>Ensure that the <span style="color: rgb(22, 145, 121);"><strong>Dgraph ApplyCh Size </strong></span>is down to 0 (zero) for the cluster</li>
</ul>

<h2>Conclusion</h2>
<p>In this Runbook, we demostrated using simple unSkript Actions we can construct a complex Runbook such as Rebuild HA Cloud Backend. You can visit <a title="unSkript Documentation" href="https://docs.unskript.com/unskript-product-documentation/" target="_blank" rel="noopener">Documentation</a> for more information about uSkript Actions.&nbsp;</p>